# PROYECTO DE REDES SOCIALES-GRUPO19

## FASE1
Para la fase inicial del análisis en Gephi, donde el objetivo es identificar actores relevantes de la red usando medidas de centralidad y transitividad se toma la decisión de no incluir ningún atributo, en los siguientes pasos del proyecto se hará la selección de atributos relevantes para la red de inversionistas.
Adicionalmente el atributo de identidad "Id" del archivo nodos y para el archivo de arcos "Source" y "Target" se estandarizara para facilitar la conexion con los arcos.

In [22]:
import pandas as pd
import re

# Definir la función de limpieza de texto
def clean_text(text):
    if isinstance(text, str):
        text = text.upper()  # Convertir a mayúsculas
        text = re.sub(r"[^A-Z0-9, ]", "", text)  # Eliminar caracteres especiales excepto comas y espacios
        text = re.sub(r"\s+", " ", text).strip()  # Eliminar espacios extra
    return text

# Cargando el archivo de Excel
archivo_path_nodos = "Nodos-original.xlsx"
archivo_path_arcos = "Arcos-original.xlsx"
df_nodos = pd.read_excel(archivo_path_nodos)
df_arcos = pd.read_excel(archivo_path_arcos)

# Aplicar la función clean_text al campo 'Id' de nodos y campos "Source y Target" de Arcos
df_nodos["Id"] = df_nodos["Id"].astype(str).apply(clean_text)
df_nodos.rename(columns={"Id": "Label"}, inplace=True) # Se cambia el nombre del id a label para que lo lea gephi
df_nodos2=df_nodos['Label']
df_arcos["Source"] = df_arcos["Source"].astype(str).apply(clean_text)
df_arcos["Target"] = df_arcos["Target"].astype(str).apply(clean_text)
df_arcos["Type"] ='directed' # Directed porque es una relación de inversión

#Descargar archivo para etapa1
df_nodos2.to_csv("nodos.csv",index=False)
df_arcos.to_csv("arcos.csv",index=False)

## FASE2
En esta fase ustedes actuarán bajo el rol de un fondo inversionista que busca comprender el comportamiento inversionista de diferentes organizaciones para mejorar la forma en la cual dispone de sus recursos en diferentes inversiones. En particular, el fondo quiere comprender cuáles son los atributos individuales de las organizaciones que son relevantes en la formación de relaciones de inversión. Para esto, se espera que implementen algoritmos de detección de comunidades y desarrollen un análisis descriptivo de los atributos de los nodos dentro y entre comunidades.Se debe entregar un notebook donde:
1. Se realice un análisis de los atributos en común entre los nodos de las comunidades. Para esto, se espera que se utilicen los algoritmos de detección de comunidades que se consideren pertinentes.
2. Se concluya al respecto para cada algoritmo seleccionado.
3. Se presente una recomendación en el contexto del problema desde el rol de una organización que busca financiación.


Es clave seleccionar atributos que permitan detectar nodos influyentes y estructuras de conexión, por lo que se propone en una primera fase seleccionar los siguientes campos:Headquarters Location, Number of Investments, Number of Lead Investments,Number of Diversity Investments, Number of Exits, Number of Lead Investors,Number of Investors, Number of Funding Rounds, Total Funding Amount Currency (in USD)

Los Posibles atributos seran homologados a:

| Nombre original                                 | Homologación en español                |
|------------------------------------------------|----------------------------------------|
| Headquarters Location                          | Ubicación de la empresa               |
| Number of Investments                          | Número de inversiones                 |
| Number of Lead Investments                     | Número de inversiones como líder      |
| Number of Diversity Investments                | Número de inversiones diversificadas  |
| Number of Exits                                | Número de salidas a bolsa             |
| Number of Lead Investors                       | Número de inversionistas principales  |
| Number of Investors                            | Número de inversionistas              |
| Number of Funding Rounds                       | Número de rondas de financiación      |
| Total Funding Amount Currency (in USD)         | Monto total de financiación (USD)     |
| Industries                                     | Industria                             |


In [ ]:
# Diccionario con los nombres nuevos
nombres_columnas = {
    "Headquarters Location": "Ubicación de la empresa",
    "Number of Investments": "Número de inversiones",
    "Number of Lead Investments": "Número de inversiones como líder",
    "Number of Diversity Investments": "Número de inversiones diversificadas",
    "Number of Exits": "Número de salidas a bolsa",
    "Number of Lead Investors": "Número de inversionistas principales",
    "Number of Investors": "Número de inversionistas",
    "Number of Funding Rounds": "Número de rondas de financiación",
    "Total Funding Amount Currency (in USD)": "Monto total de financiación (USD)",
    "Industries": "Industria"
}

# Seleccionar las columnas candidatas a atributos (excluyendo campos irrelevantes)
columnas_candidatas = [
    "Id","Industries","Headquarters Location", "Number of Investments", "Number of Lead Investments",
    "Number of Diversity Investments", "Number of Exits", "Number of Lead Investors",
    "Number of Investors", "Number of Funding Rounds", "Total Funding Amount Currency (in USD)"
]
#Seleccionando columnas 
df_nodos2=df_nodos[columnas_candidatas]

#Homologacion de campos en el DataFrame
df_nodos2.rename(columns=nombres_columnas, inplace=True)

# Conteo de valores nulos por columna candidata
conteo_nulos = df_nodos2.isnull().sum()

# Mostrar el conteo de valores nulos
print("Conteo de valores nulos por columna candidata:")
print(conteo_nulos)

Como estamos en un analisis de redes la informacion no puede ser imputada, por lo que se si encuentran registros vacios en las colunmas se deben eliminar o evaluar si se quiere dejar ese atributo

In [ ]:
# metodo de optimizacion:
# Método de Overlapping Communities:
# Método dinámico: